In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
import numpy as np
import functools as ft

def less_than(a, b):
    '''
    Assuming a and b are on adjacent vertices of
    a z-order curve, return -1 if a < b, 0 if
    a == b and 1 if a > b
    
    Parameters
    ----------
    a: (float, float)
        Point
    b: (float, float)
        Point
    '''
    
    x = np.array(a)
    y = np.array(b)
        
    ret = 1
    
    if a[0] == b[0] and a[1] == b[1]:
        return 0
    
    if a[1] < b[1]:
        ret = -1
    else:
        if a[0] < b[0]:
            ret = -1
    if a[1] > b[1]:
        ret = 1
    
    #print(ret)
    return ret
    
    print(a[1] < b[1], a[0] < b[0], a[1] > b[1], ret)
    #return True if a[1] <= b[1] else a[0] <= b[0]
    #print(a[1] <= b[1], a[0] <= b[0]
    
less_than([0,0], [0,1]) # -1 or True
less_than([0,0], [1,0]) # -1 or True
less_than([0,0], [1,1]) # -1 or True

less_than([1,0], [0,1]) # -1 or True
less_than([1,0], [1,1]) # -1 or True

less_than([0,1], [1,1]) # -1 or True
print("-------------")

less_than([0,1], [0,0]) #
less_than([1,0], [0,0]) #
less_than([1,1], [0,0]) #

less_than([0,1], [1,0]) # False
less_than([1,1], [1,0]) #

less_than([1,1], [0,1]) #

-------------


1

In [26]:
def in_tile(a, tile):
    '''
    Is a point in a tile
    
    Parameters
    ----------
    a: (float, float)
    b: (float, float)
    tile: (float, float, float, float)
    '''
    return a[0] >= tile[0] and a[1] >= tile[1] and a[0] <= tile[2] and a[1] <= tile[3]

print(in_tile([1,1], [0,0,1,1]))
print(in_tile([2,2], [0,0,1,1]))
print(in_tile([-1,1], [0,0,1,1]))
print(in_tile([0.5,0.5], [0,0,1,1]))

True
False
False
True


In [27]:
def tile_children(tile):
    '''
    Split a tile into its four children
    
    Parameters
    ----------
    tile: (float, float, float, float)
    '''
    tile_width = tile[2] - tile[0]
    tile_height = tile[3] - tile[1]
    
    return [
        [tile[0], tile[1], tile[0] + tile_width / 2, tile[1] + tile_height / 2],
        [tile[0], tile[1] + tile_height / 2, tile[0] + tile_width / 2, tile[3]],
        [tile[0] + tile_width / 2, tile[1], tile[2], tile[1] + tile_height / 2],
        [tile[0] + tile_width / 2, tile[1] + tile_height / 2, tile[2], tile[3]]
    ]
        
tile_children([0,0,1,1])

[[0, 0, 0.5, 0.5], [0, 0.5, 0.5, 1], [0.5, 0, 1, 0.5], [0.5, 0.5, 1, 1]]

In [28]:
def zindex_compare(a,b,tile):
    '''
    Compare two points along z curve wihin the bounds of 
    the given tile.
    
    Parameters
    ----------
    a: (float, float)
        Point 1
    b: (float, float)
        Point 2
    tile: (float, float, float, float)
        The bounds of the region in which we want to compare them
    '''
    #print(a, tile, in_tile(a,tile))
    assert(in_tile(a,tile))
    assert(in_tile(b, tile))
    
    children = tile_children(tile)
    
    for child in children:
        #print("child:", child)
        
        if in_tile(a, child):
            child_a = child
        if in_tile(b, child):
            child_b = child
            
    if child_a == child_b:
        #print("child_a", child_a, "child_b", child_b)
        return zindex_compare(a, b, child_a)
    else:
        return less_than(child_a[:2], child_b[:2])
        

assert(zindex_less_than([0,4], [3,4], [0,0,8,8]) == -1)
assert(zindex_less_than([6,4], [3,1], [0,0,8,8]) == 1)
assert(zindex_less_than([3,1], [2,2], [0,0,8,8]) == -1)
assert(zindex_less_than([2,2], [3,1], [0,0,8,8]) == 1)
assert(zindex_less_than([7,1], [2,2], [0,0,8,8]) == 1)
assert(zindex_less_than([2,2], [7,1], [0,0,8,8]) == -1)

In [34]:
%%time

sorted([
    [6,4],[7,1],[2,2],[3,1],[3,4],[0,4],[1,6],[4,2]
], key=ft.cmp_to_key(lambda a,b: zindex_less_than(a,b, [0,0,8,8])))

CPU times: user 192 µs, sys: 1 µs, total: 193 µs
Wall time: 207 µs


[[3, 1], [2, 2], [7, 1], [4, 2], [0, 4], [3, 4], [1, 6], [6, 4]]

In [ ]:
def all_in()